In [6]:
!pip install llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
!pip install pinecone-client
!pip install llama-index llama-index-vector-stores-pinecone

In [76]:
import pandas as pd
from llama_index.core import Document, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

# CSV 파일 로드
csv_file_path = "/Users/hyottz/Desktop/24f-houseplan/daiv_houseplan/policy_saving_sentences.csv"
df = pd.read_csv(csv_file_path)

# Document 객체 생성
documents = []
for _, row in df.iterrows():
    sentence = row["sentence"]
    index = row["index"]
    doc = Document(
        text=sentence,  # 청크 텍스트
        metadata={"doc_id": index}  # 원본 문서 전체 포함
    )
    documents.append(doc)

# Hugging Face 임베딩 모델 로드
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") #"sentence-transformers/all-mpnet-base-v2" #sentence-transformers/all-MiniLM-L6-v2", #"dunzhang/stella_en_1.5B_v5"

# VectorStoreIndex 생성
index = VectorStoreIndex.from_documents(documents,embed_model=embed_model)

In [101]:
# 리트리버 생성
retriever = index.as_retriever(verbose=True,top_k=5)

# 검색 수행
results = retriever.retrieve("전세를 알아보려고 하는데 전세 사기가 걱정돼요")

for idx, result in enumerate(results):
    doc_id = result.metadata.get("doc_id")  # 문서 ID 가져오기

    # documents 리스트에서 doc_id와 일치하는 문서 찾기
    matching_docs = [doc for doc in documents if doc.metadata.get("doc_id") == doc_id]
    if not matching_docs:  # 매칭되는 문서가 없을 경우
        print(f"결과 {idx + 1}: 매칭되는 문서가 없습니다 (문서 ID: {doc_id})")
        continue

    # 매칭된 문서에서 텍스트 가져오기
    original_text = matching_docs[0].text  # 첫 번째 매칭된 문서의 텍스트
    print(f"*정책 {idx + 1}:")
    chunk_size = 50 # 한 번에 출력할 문자 수
    print("*검색결과")
    for i in range(0, len(original_text), chunk_size):
        print(original_text[i:i+chunk_size])  # 200자씩 출력
    print(f"*유사도 점수: {result.score:.3f}")
    print("-" * 50)

*정책 1:
*검색결과
[정책 제목: '주거안정월세대출'], [정책 설명: '월세 부담으로 고민인 청년들에게 청년
1. 주거안정 월세자금 대출'], [거주지 및 소득: '아래 요건을 모두 충족하는 자
 2. (세대주) 차계약을 체결하고 임차보증금의 5% 이상을 지불한 자
대출접수일 현재 민법상 성년인 세대주(대출실행일로부터 1개월 이내에 세대분가 또는 세대합가
* 세대주의 정의: 세대별 주민등록상에 배우자, 직계존속(배
우자의 직계존속) 또는 직계 비속인 세대원으로 이루어진 세대의 세대주 (* 형제/자매는 세
2) 주민주의 배우자, 다음에 해당하는 자도 세대주로 간주
등록표상에 세대원으로 등록된 자가 대출접수일 현재 3개월 이내에 결혼으로 세대주로 예정된 
 4. (중복대출 금지)  포함한 세대원 전원이 무주택인 자
- (주택도시기금대출) 성 전세자금대출 및 주택담보대출 미이용자
년인 세대원 전원(세대가 분리된 배우자 및 자녀, 결혼예정 배우자, 배우자의 직계존속과 동
- 대를 구성하는 경우 배우자의 직계존속 포함)이 기금 대출을 이용 중이면 대출 불가
(전세자금대출 및 주택담보대출) 차주 및 배우자(결혼예정 또는 분리된 배우자 포함)가 전세
- (학자금대출) 차주 및 배우출을 이용 중이면 대출 불가
자(결혼예정 또는 분리된 배우자 포함)가 학자금대출을 이용 중이면 대출 불가(단, 대학교 
 5. (자산) 대출신청인 및 배우자의 합산 순자산 가액
이 통계청에서 발표하는 최근년도 가계금융복지조사의 ‘소득 5분위별 자산 및 부채현황’ 중 
 ※ 2024년도 기준 3값 이하(십만원 단위에서 반올림)인 자
신청인(연대입보한 경우 연대보증모두 충족하는 자
인 포함)이 한국신용정보원 신용정보관리규약에서 정하는 아래의 신용정보 및 해제 정보가 남아
2) 금융질서문란변제/대지급, 부도, 관련인 정보
그 외, 부부에 대하여 대출취급보
 7. (공공임대주택) 대출접수일 경우에는 대출 불가능
※ 대출신청 물건지가 해당 목적물인 경우 또는
 8. (우대형) 아래 중 어느 하나에 해